In [ ]:
import numpy as np
import tensorflow as tf
import math
import pandas as pd

Estimate $\mathbb {\alpha}$
$$
\alpha = \frac{ \displaystyle \sum_{ s_{g-1} < t_i \leq s_g } \delta_i }{\displaystyle \sum_{s_{g-1}<t_i \leq s_g} m_i\left(t_i-s_{g-1}\right)+
 \displaystyle\sum_{t_i>s_g} m_i\left(s_g-s_{g-1}\right)}
$$

In [ ]:
def calc_mat(data):
    '''
    Returns two matrices to be used later in alpha estimation, each matrix's
    shape equals to (#of indivuals x 5) as g = 5.
    '''
    t = data['t'].values[np.where(data['delta'].values == 1)]
    g = np.quantile(t, [0, 0.2, 0.4, 0.6, 0.8, 1])
    g[-1] = data['t'].values.max()
    t1 = data['t'].values.reshape(-1,1) - g[:-1]
    t2 = g[1:] - g[:-1]

    indicator2 = data['t'].values.reshape(-1,1) > g[1:]
    cat = tf.reduce_sum(tf.cast(indicator2, dtype=tf.float32), axis=1)
    indicator1 = tf.one_hot(tf.cast(cat, dtype=tf.int32), depth=5)

    num = indicator1
    denom = tf.add(indicator1*t1, indicator2*t2)

    return num, denom

# @tf.function
def calc_alpha(m, delta, matNum, matDenom):
  '''
  Returns estimated alpha vector with shape (5 x 1)
  '''
  num = tf.reduce_sum(matNum*delta, axis = 0)
  denom = tf.reduce_sum(matDenom*m, axis = 0)

  return tf.expand_dims(tf.divide(num, denom), 1)

$$
S_1(t ; \boldsymbol{\alpha})=1-\exp \left\{-\alpha_g\left(t-s_{g-1}\right)-\sum_{j=1}^{g-1} \alpha_j\left(s_j-s_{j-1}\right)\right\}, \text{in which}~ t \in\left(s_{g-1}, s_g\right]
$$

In [ ]:
# @tf.function
def calc_S1(alpha, matDenom): #indicator1, indicator2):
  '''
  Returns estimated S1 vector with shape (# of individuals x 1)
  '''
  x = tf.math.exp(-(matDenom @ alpha))
  return tf.clip_by_value(x, np.finfo(float).eps, 1.0)

def calc_f1(indicator1, alpha, S):
    '''
    Calculate f1
    '''
    f1 =  (indicator1 @ alpha)*S
    return tf.clip_by_value(f1, np.finfo(float).eps, 1.0)

# Negative binomial

In [ ]:
def observedLL(eta, F1, f1, phi, delta):
    '''
    Calculate observed data likelifood
    '''
    theta = np.exp(eta)
    term = 1.0/ (1.0 + phi*theta*F1)
    Sp = term**(1/phi)
    pw = (1/phi) + 1
    fp = theta*f1*(term**(pw))

    return np.sum(delta*np.log(fp) + (1 - delta)*np.log(Sp))


In [ ]:
# @tf.function
def calcM(delta, phi, theta, S1):
  F1 = 1. - S1
  term = tf.divide((1.0 + (phi*delta))*theta*S1, 1.0 + (phi*theta*F1))
  return tf.add(delta, term)

# @tf.function
def calcL1(mi, phi, eta):
  phiI = 1./phi
  theta = tf.math.exp(eta)
  term = mi*tf.math.log((phi*theta) / (1.0 + (phi*theta))) - phiI*tf.math.log(1.0 + (phi*theta))
  return -tf.reduce_mean(term)

In [ ]:
def calcNBL1(mi, phi, eta):
  phiI = (1./phi)#.astype('float32')
  theta = tf.math.exp(eta)
  term = tf.math.lgamma(mi + phiI) - tf.math.lgamma(phiI) + mi*tf.math.log(phi*theta) - mi*tf.math.log(1 + phi*theta) - (1/phi)*tf.math.log(1 + phi*theta)
  return -tf.reduce_mean(term)

# Poisson-Gamma mixture

In [ ]:
@tf.function
def calcY(phi, delta, theta, S1):
  F1 = 1. - S1
  return tf.divide((1.0 + (phi*delta))*theta, 1.0 + (phi*theta*F1))

@tf.function
def calcLogY(phi, delta, theta, S1):
  F1 = 1. - S1
  return tf.math.digamma((1./phi) + delta) + tf.math.log(phi*theta) - tf.math.log(1. + (phi*theta*F1))


# @tf.function
def calcNB2L1(phi, logY, theta, yi):
  phiI = 1./phi
  term = phiI*(logY - tf.math.log(theta) - tf.math.log(phi) - (yi/theta)) - tf.math.lgamma(phiI)
  return -tf.reduce_sum(term)